In [1]:
from pprint import pprint

import numpy as np
import pandas as pd

from tqdm.autonotebook import tqdm

from implicit.nearest_neighbours import TFIDFRecommender, BM25Recommender
from implicit.als import AlternatingLeastSquares

from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models import ImplicitItemKNNWrapperModel, RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter

C:\Users\al_po\AppData\Local\Temp\ipykernel_13628\3580575178.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


#### Uncomment to get data

In [2]:
# %%time
# !wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
# !unzip -o data_original.zip
# !rm data_original.zip

#### Load and inspect all dataframes

In [2]:
interactions_df = pd.read_csv(
    "data_original/interactions.csv",
    sep=",",
)
print(interactions_df.shape)
interactions_df.head()

(5476251, 5)


,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [3]:
# creating Interactions object 

interactions_df[Columns.Datetime], interactions_df[Columns.Weight] = interactions_df['last_watch_dt'], interactions_df['total_dur']
interactions_df.drop(['last_watch_dt', 'total_dur'], axis=1, inplace=True)
interactions = Interactions(interactions_df)

In [4]:
items_df = pd.read_csv(
    "data_original/items.csv",
    sep=",",
)
print(items_df.shape)
items_df.head()

(15963, 14)


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


#### Create function to count choosen metrics 

In [21]:
def metrics_count(interactions, models, metrics, cv, K_RECOS, n_splits=3):
    # For each fold generate train and test part of dataset
    # Then fit every model, generate recommendations and calculate metrics

    results = []

    fold_iterator = cv.split(interactions, collect_fold_stats=True)

    for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=n_splits):
        print(f"\n==================== Fold {fold_info['i_split']} ====================")
        pprint(fold_info)

        df_train = interactions.df.iloc[train_ids]
        dataset = Dataset.construct(df_train)

        df_test = interactions.df.iloc[test_ids][Columns.UserItem]
        test_users = np.unique(df_test[Columns.User])

        # Catalog is set of items that we recommend.
        # Sometimes we recommend not all items from train.
        catalog = df_train[Columns.Item].unique()

        for model_name, model in models.items():
            model.fit(dataset)
            recos = model.recommend(
                users=test_users,
                dataset=dataset,
                k=K_RECOS,
                filter_viewed=True,
            )
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )
            res = {"fold": fold_info["i_split"], "model": model_name}
            res.update(metric_values)
            results.append(res)
        
    pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg(["mean"])
    mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
    
    pivot_results = pivot_results.style \
        .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0) \
        .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
    
    
    display(pivot_results)
            
    return pivot_results
    

#### Evaluate function 

In [22]:
# Take few simple models to compare
models = {
    "random": RandomModel(random_state=42),
    "popular": PopularModel(),
    "most_raited": PopularModel(popularity="sum_weight"),
    "tfidf_k=5": ImplicitItemKNNWrapperModel(model=TFIDFRecommender(K=5)),
    "tfidf_k=10": ImplicitItemKNNWrapperModel(model=TFIDFRecommender(K=10)),
    "bm25_k=10_k1=0.05_b=0.1": ImplicitItemKNNWrapperModel(model=BM25Recommender(K=5, K1=0.05, B=0.1)),
}

# We will calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@1": Precision(k=1),
    "prec@10": Precision(k=10),
    "recall": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

K_RECOS = 10

n_splits = 3

cv = TimeRangeSplitter(
    test_size="14D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [23]:
results = metrics_count(interactions, models, metrics, cv, K_RECOS)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'end': Timestamp('2021-07-26 00:00:00', freq='14D'),
 'i_split': 0,
 'start': Timestamp('2021-07-12 00:00:00', freq='14D'),
 'test': 398993,
 'test_items': 7394,
 'test_users': 122488,
 'train': 3239125,
 'train_items': 14730,
 'train_users': 646423}


 33%|███▎      | 1/3 [01:23<02:47, 83.88s/it]


==================== Fold 1 ====================
{'end': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'i_split': 1,
 'start': Timestamp('2021-07-26 00:00:00', freq='14D'),
 'test': 458757,
 'test_items': 7711,
 'test_users': 135624,
 'train': 3892558,
 'train_items': 15085,
 'train_users': 742256}


 67%|██████▋   | 2/3 [02:58<01:30, 90.28s/it]


==================== Fold 2 ====================
{'end': Timestamp('2021-08-23 00:00:00', freq='14D'),
 'i_split': 2,
 'start': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'test': 521381,
 'test_items': 7705,
 'test_users': 151629,
 'train': 4649162,
 'train_items': 15415,
 'train_users': 850489}


100%|██████████| 3/3 [04:46<00:00, 95.39s/it]


,prec@1,prec@10,recall,novelty,serendipity
,mean,mean,mean,mean,mean
model,,,,,
random,0.000247,0.000222,0.000674,15.555717,0.000008
popular,0.097211,0.045964,0.203517,3.722852,0.000003
most_raited,0.098136,0.041627,0.187798,4.635013,0.000012
tfidf_k=5,0.092695,0.033253,0.139871,8.408621,0.000228
tfidf_k=10,0.095623,0.040675,0.173007,7.497098,0.000197
bm25_k=10_k1=0.05_b=0.1,0.118182,0.040685,0.173333,4.072555,0.000128


### Visualize Recommendation 

In [10]:
def visualize(model, inter_data, users, item_data, K_RECOS):
    dataset = Dataset.construct(inter_data)
    # get recommendations for users in dataset
    user_recs = model.recommend(
        users=users,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True
        )
    
    for user in users:
        # get history for users
        user_history = pd.merge(dataset.interactions.df[dataset.interactions.df['user_id'] == user], item_data, on='item_id')
        print(f"\n==================== User history ==================== ")
        display(user_history)
        # get recomendations with item info
        user_reco = pd.merge(user_recs[user_recs['user_id'] == user], item_data, on='item_id')
        print(f"\n==================== User recommendations ====================")
        display(user_reco)


#### Set up model, test users and items data

In [11]:
model = RandomModel(random_state=7)
dataset = Dataset.construct(interactions_df)
model.fit(dataset)
users = interactions_df.sample(3)['user_id'].to_list()
item_data = items_df[['item_id', 'title', 'genres', 'content_type', 'keywords']]

#### Analyze the results

In [12]:
visualize(model, interactions_df, users, item_data, K_RECOS)


==================== User history ==================== 


,user_id,item_id,weight,datetime,title,genres,content_type,keywords
0,256827,902,973.0,2021-04-05,Школьный стрелок,драмы,film,"женщина-режиссер, Агент под прикрытием, Буллин..."
1,256827,11340,708.0,2021-04-03,Тринадцатый этаж,"фантастика, триллеры, детективы",film,"искуственный интеллект, по роману или книге, с..."
2,256827,638,4953.0,2021-05-12,Природа женщины,для взрослых,film,"2008, италия, природа, женщины"
3,256827,544,212.0,2021-03-26,Назад в будущее. Часть 3,"приключения, вестерн, зарубежные, фантастика, ...",film,"Назад, будущее, Часть, 3, 1990, США, бандиты, ..."
4,256827,142,9.0,2021-05-12,Маша,"драмы, триллеры",film,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма..."
5,256827,2654,27.0,2021-05-12,Псы под прикрытием,"фильмы, для детей, приключения, зарубежные, се...",film,"Псы, под, прикрытием, 2018, Великобритания, ба..."
6,256827,174,53386.0,2021-05-10,Невидимка,"фантастика, триллеры",film,"убийство, эксперимент, ученый, невидимый челов..."
7,256827,4564,44.0,2021-05-12,Женщина в беде,"русские, криминал, мелодрамы",series,"Женщина, беде, 2014, Россия"
8,256827,4560,59.0,2021-05-12,Малышки на огромном сексуальном возвышении,для взрослых,film,"2016, соединенные штаты, малышки, на, огромном..."
9,256827,370,6901.0,2021-05-12,Чернобыльское кафе (на английском языке с русс...,"зарубежные, документальное",film,"Чернобыльское, кафе, английском, языке, русски..."



==================== User recommendations ====================


,user_id,item_id,score,rank,title,genres,content_type,keywords
0,256827,11370,10,1,Без изъяна,"драмы, триллеры, криминал",film,"Лондон, Англия, алмазная шахта, независимый фи..."
1,256827,2946,9,2,"Гангстер, коп и дьявол (с тифлокомментарием)","боевики, зарубежные, триллеры, криминал",film,"Гангстер, коп, дьявол, тифлокомментарием, 2019..."
2,256827,7088,8,3,Цена золота: Cкандал в американской гимнастике,"спорт, документальное",film,"Цена, золота, Cкандал, американской, гимнастик..."
3,256827,3593,7,4,Берегите женщин,"советские, комедии",series,"Берегите, женщин, 1981, СССР, новички, отцы, д..."
4,256827,8086,6,5,"Ну что, приехали?","зарубежные, семейное, комедии",film,"Ну, что, приехали, 2005, США, любовь, отцы, де..."
5,256827,10811,5,6,Отель Мумбаи: Противостояние,"боевики, драмы, историческое, триллеры",film,"отношения между мужем и женой, Мумбаи (Бомбей)..."
6,256827,8046,4,7,Сила девяти богов,"боевики, мультфильм, фэнтези, приключения",film,"2019, таиланд, сила, девяти, богов"
7,256827,3476,3,8,Террор,"ужасы, зарубежные, триллеры",film,"Террор, 1963, США"
8,256827,7123,2,9,Голос улиц,драмы,film,"отношения между братьями и сестрами, СПИД, жес..."
9,256827,10696,1,10,Назад в будущее,"фантастика, приключения, комедии",film,"башня с часами, автомобильная гонка, террорист..."



==================== User history ==================== 


...

,user_id,item_id,score,rank,title,genres,content_type,keywords
0,110281,3910,10,1,Двойная сушка,фитнес,series,"Двойная, сушка, Россия, спорт-фитнес, спорт-тр..."
1,110281,6762,9,2,Атлантик авеню,драмы,film,"2013, франция, соединенные штаты, атлантик, авеню"
2,110281,979,8,3,Братья медведи: Тайна трёх миров,"фантастика, мультфильм, приключения, комедии",film,"лес, природа, Девочка, 2017, китай, братья, ме..."
3,110281,40,7,4,Отряд особого назначения,"драмы, зарубежные, боевики, военные",film,"Отряд, особого, назначения, 2011, Франция, сил..."
4,110281,569,6,5,Порочная страсть,"драмы, триллеры",film,"менеджер, психический срыв, сокрытие, хедж-фон..."
5,110281,8108,5,6,Хозяйка пещеры,"русские, для детей, короткометражные, спорт, и...",film,"Хозяйка, пещеры, 2013, Россия, природа, путеше..."
6,110281,10180,4,7,Уличные танцы,"зарубежные, музыкальные, мелодрамы",film,"Уличные, танцы, 2010, Великобритания, друзья, ..."
7,110281,8679,3,8,Тролли: Чудесный дом,"семейное, фэнтези",film,"2016, нидерланды, тролли, чудесный, дом"
8,110281,8520,2,9,Быть Харви Вайнштейном (с тифлокомментарием),"зарубежные, документальное",film,"Быть, Харви, Вайнштейном, тифлокомментарием, 2..."
9,110281,67,1,10,Муми-дол,"советские, для детей, русские мультфильмы",series,"Муми-дол, 1980, СССР, друзья, животные, интриг..."



==================== User history ==================== 


,user_id,item_id,weight,datetime,title,genres,content_type,keywords
0,260794,838,12.0,2021-07-20,Дело «пёстрых»,"боевики, русские, криминал",film,"Дело, пёстрых, 1958, СССР"
1,260794,445,922.0,2021-07-20,Продолжение следует,"русские, детективы",series,"Продолжение, следует, 2007, Россия"



==================== User recommendations ====================


,user_id,item_id,score,rank,title,genres,content_type,keywords
0,260794,3170,10,1,Дневник Бриджит Джонс,"драмы, мелодрамы, комедии",film,"день отдыха, Лондон, Англия, алкоголь, тоска п..."
1,260794,1553,9,2,Пираты карибского моря: На краю света,"боевики, фэнтези, приключения",film,"экзотический остров, торговая компания Восточн..."
2,260794,16325,8,3,Так начиналась легенда,"биография, драмы, советские, военные, семейное",film,"Так, начиналась, легенда, 1976, СССР, Великая,..."
3,260794,13714,7,4,Паранормальное явление 3,"ужасы, триллеры",film,"родственный отношения, ведьма, продолжение, пр..."
4,260794,3398,6,5,Худеем дома с GEOPROFIT,фитнес,series,"Худеем, дома, GEOPROFIT, 2020, Россия, спорт, ..."
5,260794,405,5,6,Бегство рогатых викингов,"семейное, приключения, комедии",film,"2018, россия, бегство, рогатых, викингов"
6,260794,15864,4,7,Бомба,"драмы, исторические, боевики",series,"Бомба, 2013, Россия"
7,260794,10171,3,8,По воле божьей,"драмы, криминал",film,"жестокое обращение с детьми, Лион, Франция, 20..."
8,260794,8818,2,9,"Не бойся, я с тобой!","мюзиклы, русские, комедии",series,"Не, бойся, я, тобой, 1981, СССР"
9,260794,12904,1,10,Житие Александра Невского,"драмы, исторические",film,"Житие, Александра, Невского, 1991, СССР"
